In [2]:
import librosa
import tensorflow as tf
import numpy as np

SAVED_MODEL_PATH = "training_mix_reajusted_input.h5"
panjang_sampel = 22050


In [3]:
#preprocess the input sample function
def preprocess(file_path, jumlah=13, fft=2048, hop=512):
        # load audio file
        signal, sample_rate = librosa.load(file_path)

        if len(signal) >= panjang_sampel:
            # ensure consistency of the length of the signal
            signal = signal[:panjang_sampel]

            # extract MFCCs
            mfcc = librosa.feature.mfcc(y=signal,sr=sample_rate,n_mfcc=40)

        return mfcc.T


In [4]:
loaded=tf.keras.models.load_model(SAVED_MODEL_PATH)

In [5]:
def predict(file_path):
       
        # extract MFCC
        mfcc = preprocess(file_path)

        # we need a 4-dim array to feed to the model for prediction: (# samples, # time steps, # coefficients, 1)
        mfcc = mfcc[np.newaxis, ..., np.newaxis]
        print(mfcc.shape)

        # get the predicted label
        predictions = loaded.predict(mfcc)

        return predictions

In [6]:
res=predict("dataset/test/girl_scream-6465.mp3")
res

(1, 44, 40, 1)
1/1 [==============================] - 0s 156ms/step


array([[0.14465454, 0.539465  , 0.28417102, 0.03170936]], dtype=float32)

In [8]:
print("Prediksi Kekerasan: \n")
print("Domestik: "+str(res[0,0]*100)+"%")
print("Fisik: "+str(res[0,1]*100)+"%")
print("Seksual: "+str(res[0,2]*100)+"%")
print("Penguntitan: "+str(res[0,3]*100)+"%")

Prediksi Kekerasan: 

Domestik: 14.46545422077179%
Fisik: 53.94650101661682%
Seksual: 28.417101502418518%
Penguntitan: 3.170936182141304%
